In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import zipfile
import os

zip_path = '/content/drive/MyDrive/MLDL_repo/step5_cut/GTA5_translated.zip'
extract_path = '/content/dataset'

os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Extraction complete!")

Extraction complete!


In [ ]:
import zipfile
import os

zip_path = '/content/drive/MyDrive/MLDL_repo/Cityscapes.zip'
extract_path = '/content/dataset'

os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Extraction complete!")

Extraction complete!


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/MLDL_repo/step5_cut')

In [ ]:
!pip install -U fvcore

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61397 sha256=af775e6c020e04bd9ea883c15c98b980db1a458025179b9155a90940b5410c8e
  Stored in directory: /root/.cache/pip/wheels/65/71/95/3b8fde5c65c6e4a806e0867c1651dcc71a1cb2f3430e8f355f
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31527 sha256=423a624b4011816fda6e9cffbce117d7e390d834e98bd17fcd80693cd006a06f
  Stored in directory: /root/.cache/pip/wheels/ba/5e/16/6117f8fe7e9c0c161a795e10d94645ebcf301ccbd01f66d8ec
Successfully built fvcore iopath


In [ ]:
import re
import os
import re

def find_latest_checkpoint(checkpoint_dir):
    if not os.path.exists(checkpoint_dir):
        print(f"Checkpoint directory {checkpoint_dir} does not exist.")
        return None

    checkpoints = [f for f in os.listdir(checkpoint_dir) if f.startswith("bisenet_epoch_") and f.endswith(".pt")]
    if not checkpoints:
        print(f"No checkpoints found in {checkpoint_dir}.")
        return None

    # Extract epoch number from filename
    def extract_epoch(fname):
        match = re.search(r"bisenet_epoch_(\d+).pt", fname)
        return int(match.group(1)) if match else -1

    # Sort by epoch descending
    checkpoints.sort(key=extract_epoch, reverse=True)

    latest = os.path.join(checkpoint_dir, checkpoints[0])
    print(f" Found latest checkpoint: {latest}")
    return latest


In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torchvision import transforms


def get_train_transform():
    return A.Compose([
        A.Resize(512, 1024),
        A.Normalize(mean=(0.485, 0.456, 0.406),
                    std=(0.229, 0.224, 0.225)),
        ToTensorV2()
    ])

/usr/local/lib/python3.11/dist-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.7' (you have '2.0.6'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [ ]:
import os
import torch
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset
from torchvision import transforms as T
from PIL import Image
from datasets_custom.gta5_translated import GTA5
from models.bisenet.build_bisenet import BiSeNet
from train import train_one_epoch

def main():


    dataset_root = '/content/dataset/GTA5_translated'

    dataset_gta5 = GTA5(root=dataset_root, transform=get_train_transform())
    print(f"Full dataset size: {len(dataset_gta5)}")

    train_loader = DataLoader(dataset_gta5, batch_size=2, shuffle=True, num_workers=2)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    num_classes = 19
    base_lr = 2.5e-4
    batch_size = 2
    epochs = 50
    context_path = 'resnet18'
    checkpoint_dir = "/content/drive/MyDrive/MLDL_repo/step5_cut/checkpoints_cut"

    if not os.path.exists(checkpoint_dir):
      os.makedirs(checkpoint_dir)

    model = BiSeNet(num_classes=num_classes, context_path=context_path)
    if torch.cuda.device_count() > 1:
        print("Using", torch.cuda.device_count(), "GPUs")
        model = nn.DataParallel(model)

    model = model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    start_epoch = 0
    start_batch = 0

    latest_ckpt = find_latest_checkpoint(checkpoint_dir)
    print("latest_ckpt is:", latest_ckpt)

    if latest_ckpt:
        print(f"Restore from checkpoint: {latest_ckpt}")
        checkpoint = torch.load(latest_ckpt, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        start_epoch = checkpoint['epoch'] + 1
        start_batch = 0  # Non serve più riprendere dal batch
        print(f"Picking up from epoch {start_epoch}")
    else:
        print("No checkpoint found, start training from scratch")

    # Check Data and Labels
    for images, labels in train_loader:
        print("Images shape:", images.shape, "dtype:", images.dtype)
        print("Labels shape:", labels.shape, "dtype:", labels.dtype)
        print("Unique labels:", torch.unique(labels))
        break  # Print for the first batch only

    # Training
    for epoch in range(start_epoch, epochs):
        current_start_batch = start_batch if epoch == start_epoch else 0
        train_one_epoch(model, train_loader, optimizer, base_lr, epoch, epochs, device,
                        checkpoint_dir=checkpoint_dir, start_batch=current_start_batch)


    final_model_path = "/content/drive/MyDrive/MLDL_repo/step5_cut/final.pt"
    torch.save(model.state_dict(), final_model_path)
    print(f"Modello finale salvato in: {final_model_path}")

if __name__ == "__main__":
    main()


Full dataset size: 1750
 Found latest checkpoint: /content/drive/MyDrive/ML&DL/Progetto/MLDL_2025_main/MLDL_step5/checkpoints_2_2 (1)/bisenet_epoch_49.pt
latest_ckpt is: /content/drive/MyDrive/ML&DL/Progetto/MLDL_2025_main/MLDL_step5/checkpoints_2_2 (1)/bisenet_epoch_49.pt
Restore from checkpoint: /content/drive/MyDrive/ML&DL/Progetto/MLDL_2025_main/MLDL_step5/checkpoints_2_2 (1)/bisenet_epoch_49.pt
Picking up from epoch 50
Images shape: torch.Size([2, 3, 512, 1024]) dtype: torch.float32
Labels shape: torch.Size([2, 512, 1024]) dtype: torch.int64
Unique labels: tensor([  0,   1,   2,   3,   4,   6,   7,   8,   9,  10,  11,  13, 255])
Modello finale salvato in: /content/drive/MyDrive/ML&DL/Progetto/MLDL_2025_main/MLDL_step5/checkpoints_2_2 (1)/final.pt


In [ ]:
import os
import torch
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset
from torchvision import transforms as T
from PIL import Image
from datasets_custom.gta5_aug import GTA5
from datasets_custom.cityscapes import CityScapes
from models.bisenet.build_bisenet import BiSeNet
from train import train_one_epoch
from train import validate


#initialize model
num_classes = 19
context_path = 'resnet18'


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = BiSeNet(num_classes=num_classes, context_path=context_path).to(device)


#load model
model_path = r"/content/drive/MyDrive/MLDL_repo/step5_cut/final.pt"
model.load_state_dict(torch.load(model_path, map_location='cpu'), strict = False )

model.eval() #python built in function

# Dataset definitions and transformation
transform = transforms.Compose([
        transforms.Resize((512, 1024)),
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.485, 0.456, 0.406),
                    std=(0.229, 0.224, 0.225)),
    ])

target_transform = transforms.Compose([
        transforms.Resize((512, 1024), interpolation=transforms.InterpolationMode.NEAREST),
        transforms.PILToTensor()
    ])

dataset_root = '/content/dataset/Cityscapes/Cityspaces'

test_dataset = CityScapes(root=dataset_root, split='val', transform=transform, target_transform=target_transform)
print(f"Test dataset size: {len(test_dataset)}")
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=True, num_workers=2)


#results
best_miou, miou, per_class_ious = validate(model, test_loader, num_classes, device, best_miou=0.0)
print("\n Risultati su Cityscape Validation:")
print(f" - mIoU: {miou:.4f}")
for idx, iou in enumerate(per_class_ious):
        print(f"Classe {idx}: IoU = {iou:.4f}")



Loaded 500 images for split: val
Test dataset size: 500


🔍 Validating: 100%|██████████| 250/250 [00:30<00:00,  8.25it/s]

 Validation mIoU: 0.1029
 New best mIoU found!

 Risultati su Cityscape Validation:
 - mIoU: 0.1029
Classe 0: IoU = 0.7073
Classe 1: IoU = 0.0304
Classe 2: IoU = 0.4479
Classe 3: IoU = 0.0217
Classe 4: IoU = 0.0002
Classe 5: IoU = 0.0062
Classe 6: IoU = 0.0035
Classe 7: IoU = 0.0001
Classe 8: IoU = 0.3398
Classe 9: IoU = 0.0525
Classe 10: IoU = 0.1978
Classe 11: IoU = 0.0003
Classe 12: IoU = 0.0008
Classe 13: IoU = 0.1251
Classe 14: IoU = 0.0102
Classe 15: IoU = 0.0123
Classe 16: IoU = 0.0000
Classe 17: IoU = 0.0000
Classe 18: IoU = 0.0000
